## 第7章 数据清洗和准备

在本章中，我会讨论处理缺失数据、重复数据、字符串操作和其
它分析数据转换的⼯具。

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


/home/ubuntu/anaconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


### 7.1 处理缺失数据


对于数值数据， pandas使⽤浮点值
NaN（Not a Number）表示缺失数据。我们称其为哨兵值，可以
⽅便的检测出来：

In [5]:
string_data = pd.Series(['asdfsdf', 'wtrafda', np.nan, 'asdfew'])
string_data
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

将缺失值表示为
NA，它表示不可⽤not available。在统计应⽤中， NA数据可能
是不存在的数据或者虽然存在，但是没有观察到（例如，数据采
集中发⽣了问题）。当进⾏数据清洗以进⾏分析时，最好直接对
缺失数据进⾏分析，以判断数据采集的问题或缺失数据可能导致
的偏差。

Python内置的None值在对象数组中也可以作为NA：

In [7]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### 滤除缺失数据

过滤掉缺失数据的办法有很多种。你可以通过pandas.isnull或布
尔索引的⼿⼯⽅法，但dropna可能会更实⽤⼀些。对于⼀个
Series， dropna返回⼀个仅含⾮空数据和索引值的Series：

In [8]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

这等价于

In [9]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

⽽对于DataFrame对象，事情就有点复杂了。你可能希望丢弃全
NA或含有NA的⾏或列。 dropna默认丢弃任何含有缺失值的⾏：

In [12]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 3.2, 12.]])
cleaned = data.dropna()
print(data)
print(cleaned)

     0    1     2
0  1.0  6.5   3.0
1  1.0  NaN   NaN
2  NaN  NaN   NaN
3  NaN  3.2  12.0
     0    1    2
0  1.0  6.5  3.0


传⼊how='all'将只丢弃全为NA的那些⾏

In [14]:
data.dropna(how='all')  # 全为NA的行被丢弃

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,3.2,12.0


In [17]:
data[4] = NA
print(data)
print()
data.dropna(axis=1, how='all')

     0    1     2   4
0  1.0  6.5   3.0 NaN
1  1.0  NaN   NaN NaN
2  NaN  NaN   NaN NaN
3  NaN  3.2  12.0 NaN



,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,3.2,12.0


另⼀个滤除DataFrame⾏的问题涉及时间序列数据。假设你只想
留下⼀部分观测数据，可以⽤thresh参数实现此⽬的